## This notebook is for solving question 4 assignment 1 of the course "Optimization Methods" by Prof. Eran Treister in Ben Gurion University of the Negev
#### This notebook include not only the raw computations but the explanations and proofs as well

In [26]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

# this will limit floating numbers to 4 numbers after the point 
np.set_printoptions(precision=4)

### Section (a)

#### Definitions
We are given a matrix $A$ and a vector $b$, both are specific, numerically.
We need to write the normal equations and solve the problem using a computer. We can use built-in functions.

In [5]:
# define the given matrix A - shape is 4x3
A = np.array([[2, 1, 2], 
              [1, -2, 1], 
              [1, 2, 3], 
              [1, 1, 1]], dtype=float)

print(A.shape)

# define the given vector b - shape 4x1
b = np.array([6, 1, 5, 2])  # 1D array - shape = (4,)
b = b.reshape(-1, 1)  # 2D column vector - shape = (4, 1); the '-1' is for automatically figure out the dimension

print (b.shape)

(4, 3)
(4, 1)


#### Normal equations
The normal equations are defined as: 

$$
A^T A x = A^T b
$$

We can't assume, and we will not check whether $A^TA$ is invertible.

Thus, we are going to use numpy linalg.lstsq() function, which, as the name suggest, refers to least squares.

np.linalg.lstsq() will provide a solution, whether $A^T A$ is invertable or not; as we already proved in question 3, there's always at least one solution.
If there's not only one unique solution, np.linalg.lstsq() will return the solution x that has the smallest l2 norm - $||x||_2$.

In [9]:
# linalg.lstsq will return the least squares solution (one of many, possibly), the residuals (one residual in our case - b is only one column),
# rank of A, and something that related to the SVD form of A
x_star, residual, A_rank, _ = np.linalg.lstsq(A, b, rcond=None)  # rcond related to the SVD form of A which we haven't learned yet, so we won't talk about it here

print(f"LS Solution:\n{x_star}")

LS Solution:
[[1.7]
 [0.6]
 [0.7]]


### Section (b)

We can check whether our x_star is unique by figuring whether $A$ is full rank or not: in our case A is full rank if $rank(A)$ = 3.

If $A$ is full rank, then the solution is unique.

The reason is derived from the course notes, notes (3) page 31: if $A$ is full rank, then $A^TA$ is invertible, thus we have one normal equation, which implies one solution.

Note: we could also prove this by the properties of $A^TA$ and the info about $A$ full rank - we would need to talk about positive semi-definite and positive definite matrices.
But we assume it's ok to use the class notes.

Remember, we already have $A$ rank from np.linalg.lstsq() in the previous section, we can just print it and see if it's 3.

In [10]:
print(A_rank)

3


As we can see, $rank(A)$ is indeed 3, which means $A$ is full rank, thus, x_star is the unique solution.

Now we need to find the minimal objective (loss) value of 
$$
||Ax^* - b||_2^2.
$$
This is exactly the squared 2-norm residual: 
$$
||r(x^*)||_2^2
$$
We have this value from our calculation using np.linalg.lstsq()

Let's print it.

In [13]:
loss = residual[0]  # residual is of shape (1,)
print(f"Squared residual norm (loss): {loss}")

Squared residual norm (loss): 1.4


### Section (c)

Here we need to calculate the *vector* 
$$
r = Ax^* - b
$$

In [14]:
r = A @ x_star - b
print(r)

[[-0.6]
 [ 0.2]
 [ 0. ]
 [ 1. ]]


Let's check if $A^Tr = 0$ as requested

In [18]:
A_trsp_r = A.T @ r
print (A_trsp_r)

[[-3.33066907e-15]
 [-3.10862447e-15]
 [-3.33066907e-15]]


This is what is called, numerically zero, it's happens due floating point stuff which discussed in class.
We can get verify that A^Tr is numerically close to 0 with np.allclose if we want to

In [19]:
print(np.allclose(A_trsp_r, np.zeros_like(A_trsp_r)))

True


Is $A^T r = 0$ suprising?

No.

The solution $x^*$ satisfies the normal equations:
$$
A^⊤Ax^∗ = A^⊤b
$$

Rewriting:
$$
A^⊤(Ax^∗−b) = 0 ⇒ A^⊤r = 0
$$

It's actually have a more "wordy" sense, $r$, the residual vector, is always orthogonal to the column space of $A$, that's the core idea of LS.

### Section (d)

Here we are basically asked that, in the vector $r$, the second row (one element), will be smaller - smaller error

We can do it by putting extra importance (weight) on the second row of the system. 

As requested, $A$ and $b$ stay the same as in section (a).

So now we need to solve 
$$
argmin_x∥Ax−b∥_W^2 
$$
Where $W$ is a diagonal matrix with $MxM$ shape. $W$ is also positive definite matrix. Only the second row of $W$ will be altered (others will be 1 to not affect the other equations).

The normal equation here is: 
$$
A^TWAx = A^TWb
$$
In our case it's one equation because:

- From previous sections we found out that $A$ is full rank, thus $A^TA$ is invertible.
- $W$ is positive definite
- From both points above, we can conclude that $A^TWA$ is invertible, thus one equation
- $A^TWA$ is also squared, dimensions check: $NxM * MxM * MxN = NxM * MxN = NxN$

We can now use np.linalg.solve() which is more efficient than np.linalg.lstsq() because it not use the SVD form 
Note: linalg.solve() require that the matrix passed - in our case it's $A^TWA$ - is squared and non-singular which we already confirmed.

We will go with a trial-and-error approach here to meet the soft requirement ("let's say < 1e -3") (we could use binary search but this is overkill for our purpose and the soft requirement) 

In [28]:
for weight in [10, 100, 1000, 5000]:
    W = np.eye(4)
    W[1, 1] = weight  # second row, second column - the second entry which affect the second equation

    # left and right side of the normal equation
    left_side = A.T @ W @ A       # (A^T W A)
    right_side = A.T @ W @ b       # (A^T W b)
    x_w = np.linalg.solve(left_side, right_side)

    # Compute residual vector using the original A and b
    r = A @ x_w - b
    print(f"Weight: {weight}, |r_2| = {abs(r[1])}")

    if abs(r[1]) < 1e-3:
        print("Requirement met")
        print(f"The solution x after requirement met:\n{x_w}")
        print(f"The residual vector r after requirement met:\n{r}")
        break

Weight: 10, |r_2| = [0.0205]
Weight: 100, |r_2| = [0.0021]
Weight: 1000, |r_2| = [0.0002]
Requirement met
The solution x after requirement met:
[[1.7059]
 [0.6764]
 [0.6471]]
The residual vector r after requirement met:
[[-6.1763e-01]
 [ 2.0588e-04]
 [-1.1369e-13]
 [ 1.0294e+00]]


We can see that while we improved residual 2, other residuals got "hurt", such as residual 4, but that's makes sense.

### Section (e)

We now need to solve regularized LS, which $λ = 0.5$ and $C = I$ (Tikhonov regularization term) (other name - Ridge regression)

We want to use np.linalg.solve() so we will speak on this form of the normal equation - only one, we talked about it in class and it is written in the notes:
$$
(A^TA + λI)x = A^Tb 
$$
(because linalg.solve() takes left side and right side of a general Ax=b system)

In [27]:
λ = 0.5
n = A.shape[1]
C = np.eye(n)  # Identity matrix

left_side = A.T @ A + λ * (C.T @ C)  # Equivalent to A^T A + λ * I
right_side = A.T @ b

x_reg = np.linalg.solve(left_side, right_side)

print(f"Regularized LS solution x:\n{x_reg}")

Regularized LS solution x:
[[1.3852]
 [0.535 ]
 [0.8896]]
